In [1]:
import json

with open('/data/pj20/wiki20m/wiki20m_test.txt', 'r') as f:
    lines = f.readlines()
    

In [7]:
d = json.loads(lines[0])
print(d)

{'token': ['It', 'is', 'under', 'the', 'administration', 'of', 'Haixi', 'Mongol', 'and', 'Tibetan', 'Autonomous', 'Prefecture', '.The', 'Huatugou', 'Airport', 'is', 'located', 'in', 'Mangnai', '.'], 'h': {'name': 'huatugou airport', 'id': 'Q3207552', 'pos': [13, 15]}, 't': {'name': 'mangnai', 'id': 'Q1201941', 'pos': [18, 19]}, 'relation': 'place served by transport hub'}


In [10]:
' '.join(d['token'])

'It is under the administration of Haixi Mongol and Tibetan Autonomous Prefecture .The Huatugou Airport is located in Mangnai .'

We take 10%

In [14]:
from collections import defaultdict

text2triples = defaultdict(list)

for line in lines:
    data = json.loads(line)
    relation = data['relation']
    if relation != 'NA':
        text = ' '.join(data['token'])
        head = data['h']['name']
        tail = data['t']['name']
        text2triples[text].append((head, tail, relation))
    

In [15]:
len(text2triples)

97866

In [16]:
with open ('./wiki20m.json', 'w') as f:
    json.dump(text2triples, f, indent=6)

In [31]:
with open('../prompts/prompt_mistral.txt', 'r') as f:
    prompt = f.read()

In [32]:
prompt

'Given a patient note, extrapolate as many relationships as possible from it and provide a list of triplets. \n Provide [`ENTITY 1`, `RELATIONSHIP`, `ENTITY 2`]. The relationship is directed, so the order of entities in each triplet matters. \n\n Example 1: \n prompt: Penicillin is an antibiotic that treats bacterial infections. It was discovered by Alexander Fleming. \n relations: \n [["Penicillin", "is a type of", "antibiotic"], ["Penicillin", "treats", "bacterial infections"], ["Penicillin", "discovered by", "Alexander Fleming"]] \n\n Example 2: \n prompt: Metformin is commonly prescribed for managing type 2 diabetes. It helps by lowering glucose production in the liver and increasing the body\'s sensitivity to insulin.\n relations:\n [["Metformin", "is prescribed for", "managing type 2 diabetes"], ["Metformin", "helps by", "lowering glucose production in the liver"], ["Metformin", "increases", "body\'s sensitivity to insulin"]]\n\n prompt: $TEXT$ \n relations:\n'

In [3]:
import json
import random

wiki20m_all = json.load(open('./wiki20m_all.json', 'r'))

In [5]:
# Remove repeated values (which are lists themselves) for each pair
unique_values_dict = {key: [list(t) for t in set(tuple(item) for item in value)] for key, value in wiki20m_all.items()}

# Sort the dictionary by the size of its unique values
sorted_items = sorted(unique_values_dict.items(), key=lambda x: len(x[1]), reverse=True)

# Slice the first 500 items
top_500_items = sorted_items[:500]

# Convert back to dictionary if needed
top_500_dict = dict(top_500_items)

with open ('./wiki20m_top_500.json', 'w') as f:
    json.dump(top_500_dict, f, indent=6)


In [ ]:
# Remove repeated values (which are lists themselves) for each pair
unique_values_dict = {key: [list(t) for t in set(tuple(item) for item in value)] for key, value in wiki20m_all.items()}

# Ensure there are at least 500 items to sample from
if len(unique_values_dict) >= 500:
    # Randomly select 500 items
    random_500_items = random.sample(list(unique_values_dict.items()), 500)
else:
    print("Not enough items to select 500 unique pairs.")
    random_500_items = list(unique_values_dict.items())

# Convert back to dictionary if needed
random_500_dict = dict(random_500_items)


with open ('./wiki20m_rand_500.json', 'w') as f:
    json.dump(random_500_dict, f, indent=6)
